In [ ]:
! python tools/dataset_converters/labelme2coco.py --img-dir D:\Data\Research\Rip\rip-my\JPEGImages \
                                                --labels-dir D:\Data\Research\Rip\rip-my\LabelMe \
                                                --out data\rip_gd\annotations\all.json \
                                                

In [ ]:
# run below in terminal
! python tools/analysis_tools/browse_coco_json.py --img-dir data\rip_gd\images --ann-file data\rip_gd\annotations\test.json
                                                

In [1]:
! python tools/misc/coco_split.py --json data\rip_gd\annotations\all.json \
                                --out-dir data\rip_gd\annotations \
                                --ratios 0.7 0.15 0.15 \
                                --shuffle \
                                --seed 10


loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
Split info: ====== 
Train ratio = 0.7, number = 132
Val ratio = 0.15, number = 27
Test ratio = 0.15, number = 27
Set the global seed: 10
shuffle dataset.
Saving json to data\rip_gd\annotations\train.json
Saving json to data\rip_gd\annotations\val.json
Saving json to data\rip_gd\annotations\test.json
All done!


In [9]:
! python tools/analysis_tools/dataset_analysis.py configs/custom_dataset/yolov5_s-v61_syncbn_fast_1xb32-100e_cat.py \
                                                --out-dir work_dirs/dataset_analysis_cat/train_dataset

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


Print current running information:
+--------------------------------------------------------------------+
|                        Dataset information                         |
+---------------+-------------+--------------+-----------------------+
|  Dataset type |  Class name |   Function   |       Area rule       |
+---------------+-------------+--------------+-----------------------+
| train_dataset | All classes | All function | [0, 32, 96, 100000.0] |
+---------------+-------------+--------------+-----------------------+

Read the information of each picture in the dataset:
[                                                  ] 0/815, elapsed: 0s, ETA:
[                                 ] 1/815, inf task/s, elapsed: 0s, ETA:     0s
[                                 ] 2/815, inf task/s, elapsed: 0s, ETA:     0s
[                                 ] 3/815, inf task/s, elapsed: 0s, ETA:     0s
[          

In [14]:
! python tools/analysis_tools/optimize_anchors.py configs/custom_dataset/yolov5_s-v61_syncbn_fast_1xb32-100e_cat.py \
                                                --algorithm v5-k-means \
                                                --input-shape 1150 1150 \
                                                --prior-match-thr 4.0 \
                                                --out-dir work_dirs/dataset_analysis_cat

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
07/05 09:48:02 - mmengine - INFO - Collecting bboxes from annotation...
[                                                  ] 0/163, elapsed: 0s, ETA:
[                                 ] 1/163, inf task/s, elapsed: 0s, ETA:     0s
[                                 ] 2/163, inf task/s, elapsed: 0s, ETA:     0s
[                                 ] 3/163, inf task/s, elapsed: 0s, ETA:     0s
[                                 ] 4/163, inf task/s, elapsed: 0s, ETA:     0s
[>                                ] 5/163, inf task/s, elapsed: 0s, ETA:     0s
[>                                ] 6/163, inf task/s, elapsed: 0s, ETA:     0s
[>                                ] 7/163, inf task/s, elapsed: 0s, ETA:     0s
[>                                ] 8/163, inf task/s, elapsed: 0s, ETA:     0s
[>                                ] 9/163, inf task/s, elapsed: 0s, ETA:     0s
[>                               ] 10/163, inf 

Traceback (most recent call last):
  File "d:\Codes\Python\AI\4_Baseline\openmmlab\mmyolo-0.5.0\tools\analysis_tools\optimize_anchors.py", line 647, in <module>
    main()
  File "d:\Codes\Python\AI\4_Baseline\openmmlab\mmyolo-0.5.0\tools\analysis_tools\optimize_anchors.py", line 643, in main
    optimizer.optimize()
  File "d:\Codes\Python\AI\4_Baseline\openmmlab\mmyolo-0.5.0\tools\analysis_tools\optimize_anchors.py", line 315, in optimize
    self.save_result(anchors.tolist(), self.out_dir)
  File "d:\Codes\Python\AI\4_Baseline\openmmlab\mmyolo-0.5.0\tools\analysis_tools\optimize_anchors.py", line 202, in save_result
    dump(anchor_results, json_path)
  File "c:\ProgramData\Miniconda3\envs\pytorch\lib\site-packages\mmengine\fileio\io.py", line 931, in dump
    handler.dump_to_fileobj(obj, f, **kwargs)
  File "c:\ProgramData\Miniconda3\envs\pytorch\lib\site-packages\mmengine\fileio\handlers\json_handler.py", line 32, in dump_to_fileobj
    json.dump(obj, file, **kwargs)
  File "c:\Pr

### Coco转Labelme


In [ ]:
import json
import xml.etree.ElementTree as ET

def coco_annotation_to_labelme(coco_json_path, output_xml_path):
    # 加载COCO JSON文件
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    # 获取类别映射（假设categories是按id排序的）
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    
    # 创建LabelMe的XML根元素
    root = ET.Element('annotation')
    
    # 添加文件名
    ET.SubElement(root, 'filename').text = coco_data['images'][0]['file_name']
    
    # 添加大小信息
    size = ET.SubElement(root, 'size')
    ET.SubElement(size, 'width').text = str(coco_data['images'][0]['width'])
    ET.SubElement(size, 'height').text = str(coco_data['images'][0]['height'])
    ET.SubElement(size, 'depth').text = '3'  # 假设图像是RGB
    
    # 遍历注释，添加对象
    for annotation in coco_data['annotations']:
        object_element = ET.SubElement(root, 'object')
        
        # 类别名称
        ET.SubElement(object_element, 'name').text = categories[annotation['category_id']]
        
        # 边界框
        bndbox = ET.SubElement(object_element, 'bndbox')
        ET.SubElement(bndbox, 'xmin').text = str(annotation['bbox'][0])
        ET.SubElement(bndbox, 'ymin').text = str(annotation['bbox'][1])
        ET.SubElement(bndbox, 'xmax').text = str(annotation['bbox'][0] + annotation['bbox'][2])
        ET.SubElement(bndbox, 'ymax').text = str(annotation['bbox'][1] + annotation['bbox'][3])
    
    # 将XML树写入文件
    tree = ET.ElementTree(root)
    tree.write(output_xml_path)

# 使用函数
coco_json_path = 'path/to/your/coco_annotations.json'
output_xml_path = 'path/to/output/labelme_annotation.xml'
coco_annotation_to_labelme(coco_json_path, output_xml_path)

In [1]:
import json
import xml.etree.ElementTree as ET

def polygon_from_segmentation(segmentation):
    """
    从COCO的segmentation数据生成Polygon的点列表。
    注意：COCO的segmentation可以是多边形或编码的RLE，这里仅处理多边形。
    """
    if isinstance(segmentation, list):  # 多边形格式
        return segmentation
    else:  # 如果直接处理RLE等其他格式，这里可能需要额外的逻辑
        raise NotImplementedError("Only polygon segmentation is supported in this example.")

def coco_annotation_to_labelme(coco_json_path, output_xml_path):
    with open(coco_json_path, 'r') as f:
        coco_data = json.load(f)
    
    categories = {cat['id']: cat['name'] for cat in coco_data['categories']}
    
    root = ET.Element('annotation')
    
    image_info = coco_data['images'][0]
    ET.SubElement(root, 'filename').text = image_info['file_name']
    size = ET.SubElement(root, 'size')
    ET.SubElement(size, 'width').text = str(image_info['width'])
    ET.SubElement(size, 'height').text = str(image_info['height'])
    ET.SubElement(size, 'depth').text = '3'
    
    for annotation in coco_data['annotations']:
        object_element = ET.SubElement(root, 'object')
        ET.SubElement(object_element, 'name').text = categories[annotation['category_id']]
        
        bndbox = ET.SubElement(object_element, 'bndbox')
        bbox = annotation['bbox']
        ET.SubElement(bndbox, 'xmin').text = str(bbox[0])
        ET.SubElement(bndbox, 'ymin').text = str(bbox[1])
        ET.SubElement(bndbox, 'xmax').text = str(bbox[0] + bbox[2])
        ET.SubElement(bndbox, 'ymax').text = str(bbox[1] + bbox[3])
        
        # 添加分割信息
        segmentation = polygon_from_segmentation(annotation['segmentation'])
        seg_element = ET.SubElement(object_element, 'segmentation')
        for point in segmentation:
            seg_element.text += str(point) + ' '
        seg_element.text = seg_element.text.rstrip()  # 移除尾部空格
    
    tree = ET.ElementTree(root)
    tree.write(output_xml_path, encoding='utf-8', xml_declaration=True)

# 调用函数
coco_json_path = 'E:/Data/Research/RipSeg/TrainData/train/_annotations.coco.json'
output_xml_path = 'E:/Data/Research/RipSeg/TrainData/train/labelme'
coco_annotation_to_labelme(coco_json_path, output_xml_path)

AttributeError: 'NoneType' object has no attribute 'rstrip'

In [3]:
# -*- coding: utf-8 -*-
import glob
import os
import cv2
import json
import io
import sys
args = sys.argv

print(" [Info]_coco to labelme",flush=True) 

# labelme_path = args[1]
# coco_json_path = args[2]
# jpg_path = args[3]
labelme_path = 'E:/Data/Research/RipSeg/TestData/train/labelme'
coco_json_path = 'E:/Data/Research/RipSeg/TestData/train/coco'
jpg_path = 'E:/Data/Research/RipSeg/TestData/train/'
i_path = 4
class_list = []
for i in range(len(args)) :
    if i_path < len(args): 
        class_list.append(args[i_path])
        i_path = i_path + 1     
coco = str(class_list)
'''
labelme_path='C:/Users/qiaomengyu/Desktop/testdata/fengetest/mask/'
coco_json_path = 'C:/Users/qiaomengyu/Desktop/testdata/fengetest/'
jpg_path='C:/Users/qiaomengyu/Desktop/testdata/fengetest/image'

coco=["ship", "sar_ship", "plane"]'''
label=dict()
for idx,item in enumerate(coco):
    label.update({idx:item})

coco_json=glob.glob(os.path.join(coco_json_path,"*.json"))[0]
file_json = io.open(coco_json,'r',encoding='utf-8')
m_json_data = file_json.read()
m_data = json.loads(m_json_data)
#m_type=m_data['type']
jishu = 0
jindu_int = 0
jindu_flag = 0

 
for item in m_data['images']:
    jindu_int = int((jishu/int(len(m_data['images'])))*100)
    if jindu_int > jindu_flag:
        print('[Percent]_'+str(jindu_int)+'%',flush=True)
        jindu_flag = jindu_int
    jishu = jishu +1
    flag=False
    m_images_file_name = item['file_name']
    (filename_path, m_filename) = os.path.split(m_images_file_name)
    (m_name, extension) = os.path.splitext(m_filename)
    m_image=cv2.imread(os.path.join(jpg_path,m_name+".jpg"))
    m_images_height = item['height']
    m_images_width = item['width']
    m_images_id = item['id']
    data = {}
    data['imagePath'] = m_filename
    data['flags'] = {}
    data['imageWidth'] = m_images_width
    data['imageHeight'] = m_images_height
    data['imageData'] = None
    data['version'] = "5.0.1"
    data["shapes"] = []
    for annit in m_data['annotations']:
        m_image_id=annit['image_id']
        m_category_id=annit['category_id']
        if m_image_id==m_images_id:
            flag = True
            for segitem in annit['segmentation']:
                points = []
                for idx in range(0,len(segitem),2):
                    x,y=segitem[idx],segitem[idx+1]
                    if str(x).isalpha() or str(y).isalpha():
                        flag = False
                        break
                    points.append([x,y])
                itemData = {'points': []}
                if len(points)==0:
                    flag = False
                    break
                itemData['points'].extend(points)
                itemData["flag"] = {}
                itemData["group_id"] = None
                itemData["shape_type"] = "polygon"
                itemData["label"] =label[m_category_id-1]
                data["shapes"].append(itemData)
    
    if flag:
        jsonName = ".".join([m_name, "json"])
        jpgName = ".".join([m_name, "jpg"])
        #print(labelme_path, jsonName)
        jsonPath = os.path.join(labelme_path, jsonName)
        jpgPath = os.path.join(labelme_path, jpgName)
        with open(jsonPath, "w") as f:
            json.dump(data, f)
        cv2.imwrite(jpgPath,m_image)
        #print("加载入文件完成...")
print('[Percent]_100%',flush=True)


 [Info]_coco to labelme
[Percent]_1%
[Percent]_2%
[Percent]_3%
[Percent]_4%
[Percent]_5%
[Percent]_6%
[Percent]_7%
[Percent]_8%
[Percent]_9%
[Percent]_10%
[Percent]_11%
[Percent]_12%
[Percent]_13%
[Percent]_14%
[Percent]_15%
[Percent]_16%
[Percent]_17%
[Percent]_18%
[Percent]_19%
[Percent]_20%
[Percent]_21%
[Percent]_22%
[Percent]_23%
[Percent]_24%
[Percent]_25%
[Percent]_26%
[Percent]_27%
[Percent]_28%
[Percent]_29%
[Percent]_30%
[Percent]_31%
[Percent]_32%
[Percent]_33%
[Percent]_34%
[Percent]_35%
[Percent]_36%
[Percent]_37%
[Percent]_38%
[Percent]_39%
[Percent]_40%
[Percent]_41%
[Percent]_42%
[Percent]_43%
[Percent]_44%
[Percent]_45%
[Percent]_46%
[Percent]_47%
[Percent]_48%
[Percent]_49%
[Percent]_50%
[Percent]_51%
[Percent]_52%
[Percent]_53%
[Percent]_54%
[Percent]_55%
[Percent]_56%
[Percent]_57%
[Percent]_58%
[Percent]_59%
[Percent]_60%
[Percent]_61%
[Percent]_62%
[Percent]_63%
[Percent]_64%
[Percent]_65%
[Percent]_66%
[Percent]_67%
[Percent]_68%
[Percent]_69%
[Percent]_70%
[Perc